## Exploring the spatial distribution of the gap between aging rate and PTAL

#### Created by Tomohiro Ujikawa 
##### (Created in Apr/2025)
- Graduate Student, Luskin School of Public Affairs, University of California, Los Angeles
- Project Researcher, Center for Preventive Medical Sciences, Chiba University

---

#### Data Used

1. PTAL data for Sumida Ward created in the `PTAL_Sumida` notebook (`mesh_merged`)
2. 2020 Census population data by age group at the small-area (town/block) level  
   [2020 Census → Small Area Tabulation → Tokyo → Population by Sex and 5-Year Age Group, Average Age, and Total Age by Town/Block](https://www.e-stat.go.jp/stat-search/files?tclass=000001147861&cycle=0)
3. 2020 Census boundary shapefile at the small area level  
   [Boundary Data → Small Area → 2020 Census → Town/Block Boundaries (JGD2000) → WGS84/Longitude-Latitude → Tokyo → Sumida Ward](https://www.e-stat.go.jp/gis/statmap-search?page=1&type=2&aggregateUnitForBoundary=A&toukeiCode=00200521&toukeiYear=2020&serveyId=A002005212020&prefCode=13&coordsys=1&format=shape&datum=2000)

---

#### Calculate aging rate at the small area level within Sumida City and convert to GeoDataFrame

In [1]:
import os
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm
from scipy.stats import zscore

In [2]:
# Load census data on elderly population at the town/block level in Tokyo
census = pd.read_csv('data/Census2020_Age_Tokyo.csv')

# Load boundary shapefile for the 2020 Census
boundary = gpd.read_file("data/Sumida_Chouchouaza_Census2020.zip")

In [3]:
# Filter for Sumida Ward only (municipality code: 13107)
census = census[census["市区町村コード"] == 13107]

In [4]:
# Remove gender-specific rows and keep only the total population
census = census[census["男女"] == "総数"]

# Remove the aggregated total row for the entire ward
census = census[census["町丁字コード"] != "-"]

In [5]:
# To merge, use the "町丁字コード" from census and "S_AREA" from boundary

# First, convert "町丁字コード" to string and zero-pad to 6 digits
census["町丁字コード"] = census["町丁字コード"].astype(str).str.zfill(6)

# Also convert "S_AREA" in boundary to string and zero-pad
boundary["S_AREA"] = boundary["S_AREA"].astype(str).str.zfill(6)

In [6]:
# Perform a left merge using boundary as the base (since census includes both raw and aggregated rows)
agingGdf = boundary.merge(census, left_on="S_AREA", right_on="町丁字コード", how="left")

In [7]:
# Rename the column "(Reposted) Age 65 and over" to English
agingGdf = agingGdf.rename(columns={'（再掲）65歳以上': 'pop_over65'})

# Calculate aging rate: proportion of population age 65 and over
agingGdf['pc_over65'] = agingGdf['pop_over65'].astype(float) / agingGdf['総数'].astype(float) * 100

#### Mapping the aging rate within Sumida City

In [8]:
# Convert CRS to WGS84 for Folium visualization
agingGdf = agingGdf.to_crs(epsg=4326)

# Calculate the map center based on geometry centroids
centroid = agingGdf.geometry.centroid
center_lat = centroid.y.mean()
center_lon = centroid.x.mean()

# Create a Folium map centered on Sumida Ward
m2 = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# Get the min and max values of aging rate (%)
min_val = agingGdf['pc_over65'].min()
max_val = agingGdf['pc_over65'].max()

# Define color map (reversed Spectral scheme)
original_colormap = cm.linear.Spectral_11
reversed_colors = list(original_colormap.colors)[::-1]
colormap = cm.LinearColormap(reversed_colors, vmin=min_val, vmax=max_val)
colormap.caption = 'Proportion of people over 65 (%)'

# Draw the GeoDataFrame on the map
folium.GeoJson(
    agingGdf,
    name="Aging Layer",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['pc_over65']),
        'color': 'None',  # No border lines
        'weight': 0,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['pc_over65'])
).add_to(m2)

# Add the color map legend
colormap.add_to(m2)

# Save the map as an HTML file
m2.save("pc_over65_map.html")
m2

/var/folders/lx/_3z8wwyx3ds15_0k28hby_h80000gn/T/ipykernel_18094/3690497012.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = agingGdf.geometry.centroid


#### Combine PTAL's Access Index data with aging rate data

In [10]:
# Load Access Index data
mesh_merged = gpd.read_file("../created_data/mesh_merged.gpkg")

In [11]:
# Align CRS of both GeoDataFrames (match to agingGdf)
mesh_merged = mesh_merged.to_crs(agingGdf.crs)

In [12]:
# Perform a spatial join to assign each mesh to its corresponding town/block
joined = gpd.sjoin(agingGdf[["S_AREA", "geometry"]], mesh_merged[["AI", "geometry"]], how="inner", predicate="intersects")

In [13]:
# Calculate average AI for each town/block (S_AREA)
chouchou_AI = joined.groupby("S_AREA")["AI"].mean().reset_index()

In [14]:
# Merge the averaged AI back into agingGdf
agingGdf = agingGdf.reset_index()
agingGdf = agingGdf.merge(chouchou_AI, left_on="S_AREA", right_on="S_AREA", how="left")

##### Convert both aging rate and AI into Z-scores and calculate a gap index

In [15]:
agingGdf["pc_over65_z"] = zscore(agingGdf["pc_over65"])
agingGdf["AI_z"] = zscore(agingGdf["AI"])
agingGdf["gap_z"] = agingGdf["pc_over65_z"] - agingGdf["AI_z"]

##### Mapping the gap index

In [16]:
# Convert CRS to WGS84
agingGdf = agingGdf.to_crs(epsg=4326)

# Calculate map center based on geometry centroids
centroid = agingGdf.geometry.centroid
center_lat = centroid.y.mean()
center_lon = centroid.x.mean()

# Create a Folium map
m_gap = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# Get min and max values for gap_z (clipped to [-3, 3])
min_val = max(-3, agingGdf['gap_z'].min())
max_val = min(3, agingGdf['gap_z'].max())

# Reverse Spectral_11 colormap
original_colormap = cm.linear.Spectral_11
reversed_colors = list(original_colormap.colors)[::-1]
colormap = cm.LinearColormap(reversed_colors, vmin=min_val, vmax=max_val)
colormap.caption = 'Gap Index (High Aging – Low Access)'

# Visualize using GeoJson
folium.GeoJson(
    agingGdf,
    name="Gap Z Layer",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['gap_z']),
        'color': None,  # No border line
        'weight': 0,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['gap_z'])
).add_to(m_gap)

# Add colormap legend
colormap.add_to(m_gap)

# Save map to HTML
m_gap.save("gap_z_map.html")
m_gap

/var/folders/lx/_3z8wwyx3ds15_0k28hby_h80000gn/T/ipykernel_18094/346625288.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = agingGdf.geometry.centroid


##### Create an index to highlight areas with greater investment needs by multiplying by older population

In [17]:
agingGdf["investment_priority"] = agingGdf["gap_z"] * agingGdf['pop_over65'].astype(float)

##### Mapping areas with higher investment priority

In [18]:
# Convert CRS to WGS84
agingGdf = agingGdf.to_crs(epsg=4326)

# Calculate center of the map
centroid = agingGdf.geometry.centroid
center_lat = centroid.y.mean()
center_lon = centroid.x.mean()

# Create Folium map
m_priority = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# Get min and max values of investment_priority
min_val = agingGdf["investment_priority"].min()
max_val = agingGdf["investment_priority"].max()

# Use reversed Spectral_11 colormap
original_colormap = cm.linear.Spectral_11
reversed_colors = list(original_colormap.colors)[::-1]
colormap = cm.LinearColormap(reversed_colors, vmin=min_val, vmax=max_val)
colormap.caption = 'Investment Priority Score (High Aging x Low Access × Older Population Size)'

# Draw using GeoJson
folium.GeoJson(
    agingGdf,
    name="Investment Priority",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['investment_priority']),
        'color': None,  # No border lines
        'weight': 0,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['investment_priority', 'gap_z', 'pop_over65'])
).add_to(m_priority)

# Add colormap legend
colormap.add_to(m_priority)

# Save map to HTML
m_priority.save("investment_priority.html")
m_priority

/var/folders/lx/_3z8wwyx3ds15_0k28hby_h80000gn/T/ipykernel_18094/1281987137.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = agingGdf.geometry.centroid


In [ ]:
# Save the aging rate GeoDataFrame to a GPKG file
agingGdf.to_file("../created_data/aging.gpkg", driver="GPKG")

#### Key Findings

* Areas with higher aging rates (indicating greater need for public transportation) generally have lower Access Index (AI) scores, suggesting limited public transportation supply.
* These areas can be identified as having high demand for public transit improvements.
* Such areas are predominantly concentrated in the northern part of the ward, indicating a north-south disparity.  
<br>
* For these high-priority areas, it is recommended to focus on:
    * Enhancing the operation of community buses 
    * Introducing pilot projects
    * Assessing the availability of walkable services (e.g., healthcare, food access)
    * Evaluating the quality and safety of the walking environment